In [18]:
%run functions.ipynb

In [19]:
class learn():
    
    pred_col = '_predicted'
    def __init__(self,features,target,cv):
        
        self.features = features
        self.target = target
        
        df = features.join(target)

        self.cv = cv
        self.pred_col
        
        from sklearn.model_selection import train_test_split
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.features, self.target, test_size=0.2, random_state=41234)
        
        df_norm = normalize(df)
        self.features_norm = df_norm[features.columns]
        self.target_norm = df_norm[target.columns]
        self.X_train_norm, self.X_test_norm, self.y_train_norm, self.y_test_norm = train_test_split(self.features_norm, self.target_norm, test_size=0.2, random_state=41234)

        
    
    def linear(self,cross_validate=True):
        
        from sklearn import linear_model
        reg = linear_model.LinearRegression()
        

        if cross_validate == True:
            predictions_frame = cross_val_regression(reg,self.features,self.target,self.pred_col,self.cv)
            
        else:
            predictions_frame = train_test_regression(reg,self.X_train,self.X_test,self.y_train,self.y_test,self.pred_col)
            
        coef_v, mbe, r2 = get_metrics(predictions_frame)
            
        return predictions_frame, coef_v , mbe, r2
    
    def rdf(self,cross_validate=True):
        
        from sklearn.ensemble import RandomForestRegressor
        reg = RandomForestRegressor()
        
        if cross_validate == True:
            predictions_frame = cross_val_regression(reg,self.features,self.target,self.pred_col,self.cv)
            
        else:
            predictions_frame = train_test_regression(reg,self.X_train,self.X_test,self.y_train,self.y_test,self.pred_col)
       
        coef_v, mbe, r2 = get_metrics(predictions_frame)
            
        return predictions_frame, coef_v , mbe, r2
    
    def svm(self,cross_validate=True):
        
        from sklearn.svm import SVR
        reg = SVR(kernel='rbf')
        
        if cross_validate == True:
            predictions_frame = cross_val_regression(reg,self.features_norm,self.target_norm,self.pred_col,self.cv)
            
        else:
            predictions_frame = train_test_regression(reg,self.X_train_norm,self.X_test_norm,self.y_train_norm,self.y_test_norm,self.pred_col)
            
        #denormalize the dataframe
        predictions_frame_denorm = (predictions_frame*(self.target.max()[0]-self.target.min()[0]))+self.target.min()[0]
       
        coef_v, mbe, r2 = get_metrics(predictions_frame_denorm)
            
        return predictions_frame_denorm, coef_v , mbe, r2
            

        

In [ ]:
def cross_val_regression(reg,features,target,pred_col,cv):
    
    from sklearn.model_selection import cross_val_predict
    
    predict = cross_val_predict(reg, features, target,cv=cv)
    predictions = pd.Series(predict.ravel(),index=target.index).rename(target.columns[0]+pred_col)
    predictions_frame = pd.DataFrame(predictions).join(target).dropna()
    
    return predictions_frame

def train_test_regression(reg, X_train, X_test, y_train, y_test,pred_col):
    
    reg.fit(X_train, y_train)
    predict = reg.predict(X_test)
    predictions = pd.Series(predict.ravel(),index=y_test.index).rename(y_test.columns[0]+pred_col)
    predictions_frame = pd.DataFrame(predictions).join(y_test).dropna()
    
    return predictions_frame
        
    
    